In [1]:
import os
from dotenv import load_dotenv
try:
    load_dotenv(dotenv_path='/home/connectivity-checker/.env', override=True)
except:
    try:
        load_dotenv()
    except:
        pass
    
from utils.logging import Logging
from utils.mq import MQ
from utils.bigquery import BigQuery

logger = Logging(__name__).init()
mq = MQ().from_json_config(os.environ.get('CONFIG_MQ'), 'ASM')
bigquery_dml = BigQuery().connect(os.environ.get('SECRET_GSERVICE_DML'))

from sqls.asm_con import sql

r = bigquery_dml.select_rows_by_standard_sql(sql.get_select_msg_payload('dml-prod'))

mq.set_message(r['payload'][0])
mq.set_correlation_id(r['correlationId'][0])


[thread_id 139824471252992] 2024-05-27 12:06:37,279 - __main__ - INFO - Initalize...
[thread_id 139824471252992] 2024-05-27 12:06:37,290 - utils.bigquery - INFO - Establish connection: dml-prod
[thread_id 139824471252992] 2024-05-27 12:06:37,433 - utils.bigquery - INFO - Session created. Dml-Prod
[thread_id 139824471252992] 2024-05-27 12:06:37,441 - utils.bigquery - INFO - Execute SQL on Dml-Prod:

    -- Author: hexton.chan@hkexpress.com 
    -- Description: Get 7-day flight schedule IN which updated BY 7-day before departure, generate ASM_CON
  SELECT
    ID,
    FLTID,
    DATOP_CHN,
    'OpmLegChangeAsmCon-' || FLTID || UPPER(FORMAT_DATETIME("-%y%m%d%H%M%S",SRC_UPDATED_TIME))  AS correlationId,
    'ASM' AS msgType,
    --'UTC' AS timeZone,
    --UPPER(FORMAT_DATETIME("%d%b%y",CURRENT_DATETIME())) AS msgGenTime,
    --'CON' AS msgSubType,
    --FLIGHT_NO,
    --UPPER(FORMAT_DATETIME("%d%b%y",DATOP)) AS DATOP,
    --ACTYPE,
    --LONG_REG,
    --SERVICE_TYPE_ID,
    --DEPSTN,
    --

In [2]:
mq.callback()

[thread_id 139663934713856] 2024-05-24 18:14:38,740 - utils.mq - INFO - ==== [START] MQ Java Subprocess ====
[thread_id 139663934713856] 2024-05-24 18:14:42,155 - utils.mq - INFO - [Success 0] Message: OpmLegChangeAsmCon-211255-240523142740 successfully sent to remote side.
[thread_id 139663934713856] 2024-05-24 18:14:42,157 - utils.mq - CRITICAL - failed to get connect or send message
com.ibm.msg.client.jms.DetailedJMSException: JMSWMQ0018: Failed to connect to queue manager 'MQEXTT01' with connection mode 'Client' and host name 'externalmq-t0.ete.cathaypacific.com(1414)'.
Check the queue manager is started and if running in client mode, check there is a listener running. Please see the linked exception for more information.
	at com.ibm.msg.client.wmq.common.internal.Reason.reasonToException(Reason.java:595)
	at com.ibm.msg.client.wmq.common.internal.Reason.createException(Reason.java:215)
	at com.ibm.msg.client.wmq.internal.WMQConnection.<init>(WMQConnection.java:448)
	at com.ibm.msg

AssertionError: ('utils.mq', '[Error 1] Unknown Exception when calling MQ Java.\nDEBUG - <class \'AssertionError\'> (\'utils.mq\', "failed to get connect or send message\\ncom.ibm.msg.client.jms.DetailedJMSException: JMSWMQ0018: Failed to connect to queue manager \'MQEXTT01\' with connection mode \'Client\' and host name \'externalmq-t0.ete.cathaypacific.com(1414)\'.\\nCheck the queue manager is started and if running in client mode, check there is a listener running. Please see the linked exception for more information.\\n\\tat com.ibm.msg.client.wmq.common.internal.Reason.reasonToException(Reason.java:595)\\n\\tat com.ibm.msg.client.wmq.common.internal.Reason.createException(Reason.java:215)\\n\\tat com.ibm.msg.client.wmq.internal.WMQConnection.<init>(WMQConnection.java:448)\\n\\tat com.ibm.msg.client.wmq.factories.WMQConnectionFactory.createV7ProviderConnection(WMQConnectionFactory.java:8475)\\n\\tat com.ibm.msg.client.wmq.factories.WMQConnectionFactory.createProviderConnection(WMQC